In [87]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [88]:
# Replace 'MSFT' with your desired ticker
ticker_symbol = 'GLD'

ticker_data = yf.Ticker(ticker_symbol)

# Get historical market data
hist_data = yf.download(ticker_symbol, start=datetime.strptime("2022-11-18", '%Y-%m-%d'), end=datetime.now(), interval="1d")

print(hist_data)


[*********************100%%**********************]  1 of 1 completed

                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2022-11-18  163.380005  163.690002  162.679993  162.789993  162.789993   
2022-11-21  162.440002  162.500000  161.279999  161.880005  161.880005   
2022-11-22  162.300003  162.600006  161.720001  162.070007  162.070007   
2022-11-23  161.690002  163.240005  161.559998  163.080002  163.080002   
2022-11-25  162.850006  163.429993  162.759995  163.220001  163.220001   
...                ...         ...         ...         ...         ...   
2024-05-20  223.789993  225.660004  223.020004  224.559998  224.559998   
2024-05-21  224.770004  225.210007  223.699997  224.229996  224.229996   
2024-05-22  223.089996  223.240005  219.740005  220.110001  220.110001   
2024-05-23  218.779999  219.229996  215.660004  215.720001  215.720001   
2024-05-24  216.960007  217.210007  215.740005  215.919998  215.919998   

             Volume  
Date           

In [89]:
hist_data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-11-18,163.380005,163.690002,162.679993,162.789993,162.789993,4321100
2022-11-21,162.440002,162.500000,161.279999,161.880005,161.880005,4766500
2022-11-22,162.300003,162.600006,161.720001,162.070007,162.070007,3203800
2022-11-23,161.690002,163.240005,161.559998,163.080002,163.080002,4167000
2022-11-25,162.850006,163.429993,162.759995,163.220001,163.220001,1436500
...,...,...,...,...,...,...
2024-05-20,223.789993,225.660004,223.020004,224.559998,224.559998,5990900
2024-05-21,224.770004,225.210007,223.699997,224.229996,224.229996,4002600
2024-05-22,223.089996,223.240005,219.740005,220.110001,220.110001,8903100


In [83]:
# Write to CSV
hist_data.to_csv('gold_prices.csv')

In [91]:
# Function to download data in allowed batches
def download_data(ticker, days_back=400, interval='60m', date_string="2022-11-18"):
    data_frames = []
    end_date = datetime.now()
    # start_date = end_date - timedelta(days=days_back)
    start_date = datetime.strptime(date_string, '%Y-%m-%d')
    
    # Yahoo Finance allows only 7 days of minute data
    while start_date < end_date:
        current_end_date = min(start_date + timedelta(days=7), end_date)
        data = yf.download(ticker, start=start_date, end=current_end_date, interval=interval)
        data_frames.append(data)
        start_date = current_end_date

    return pd.concat(data_frames)
    
ticker = 'GLD'
data = download_data(ticker)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [65]:
# import yfinance as yf

# # Replace 'GLD' with the appropriate ticker for your asset if needed
# ticker = 'GLD'
# data = yf.download(tickers=ticker, period='7d', interval='1m')
# data.to_csv("gold_min.csv")
# print(data.head())


In [92]:
data['Log mid-quote'] = np.log(data['Close'])
data['squared_returns'] = (data['Log mid-quote'].diff())**2
realized_volatility = data['squared_returns'].resample('D').sum()
realized_volatility = np.sqrt(realized_volatility * 257)
vol = pd.DataFrame(realized_volatility).rename(columns={"squared_returns": "Realized_Volatility"})


In [95]:
pd.merge(hist_data, vol, left_index=True, right_index=True, how="left").to_csv('gold_prices_updated.csv')